# Query

![Elasticsearch](https://storage.googleapis.com/rg-ai-bootcamp/database/elasticsearch-min.png)

_Elasticsearch (Source: [technocratsid.com](https://www.technocratsid.com/how-to-create-an-elasticsearch-6-4-1-plugin/))_

Remember when we learned to use Elasticsearch's Search API to retrieve data from an Index? But what if we want to fine-tune our search? Well, that's where queries come in. Queries are the precise tools we can use with the Search API to hone in on the data we're after.

Before we start practicing, install the elasticsearch package

In [ ]:
%pip install elasticsearch jsonlines

import the packages we need

In [ ]:
from elasticsearch import Elasticsearch, helpers
import time
import json
import jsonlines

Create a connection to elasticsearch, make sure elasticsearch is running on your computer's localhost or is running on Google Collab.

In [ ]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

Creating an **Customer Index**

In [ ]:
# Create index customers
# PUT http://localhost:9200/customers
response = es.indices.create(index='customers', ignore=400)
print(json.dumps(response.body, indent=4))

C:\Users\aditi\AppData\Local\Temp\ipykernel_35632\1248388089.py:3: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response = es.indices.create(index='customers', ignore=400)


{
    "acknowledged": true,
    "shards_acknowledged": true,
    "index": "customers"
}


Insert Customer Data with Hobbies and Banks

In [ ]:
# bulk insert customers
# POST http://localhost:9200/_bulk

# < customers.json
def load_json(file_name):
    with jsonlines.open(file_name) as reader:
        for obj in reader:
            yield {
                "_index": "customers",
                "_source": obj,
            }

helpers.bulk(es, load_json('./data/customers-with-banks.json'))

(2000, [])

### Match All

When using Elasticsearch's Search API, you need a way to retrieve all data from an index without specifying any query. By default, the Search API uses the Match All query. This query is designed to retrieve all data in an index, providing a simple and efficient way to obtain a complete set of data.

In [ ]:
# search using match_all
# POST http://localhost:9200/customers/_search
response = es.search(index='customers', body={
    'query': {
        'match_all': {}
    }
})
print(json.dumps(response.body, indent=4))

{
    "took": 3,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 2000,
            "relation": "eq"
        },
        "max_score": 1.0,
        "hits": [
            {
                "_index": "customers",
                "_id": "KbJGYowBtsOVXdO2r74I",
                "_score": 1.0,
                "_source": {
                    "index": {
                        "_index": "customers",
                        "_id": "username1"
                    }
                }
            },
            {
                "_index": "customers",
                "_id": "KrJGYowBtsOVXdO2r74I",
                "_score": 1.0,
                "_source": {
                    "username": "username1",
                    "first_name": "Rollie",
                    "last_name": "Farge",
                    "email": "rfarge0@sphinn.com",
                    "gende

### Paging dan Sorting

You need to implement paging and sorting when using the Search API, but using Query Parameters can be complex and hard to read? You can simplify this process by using the Request Body for paging and sorting. It's a more straightforward and readable approach than using Query Parameters.

In [ ]:
# search using match_all with paging and sorting
# POST http://localhost:9200/customers/_search
response = es.search(index='customers', body={
    'query': {
        'match_all': {}
    },
    'size': 10,
    'from': 0,
    'sort': [
        {
            "username.keyword": {
                "order": "desc"
            }
        }
    ]
})
print(json.dumps(response.body, indent=4))

{
    "took": 4,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 2000,
            "relation": "eq"
        },
        "max_score": null,
        "hits": [
            {
                "_index": "customers",
                "_id": "9rJGYowBtsOVXdO2sMWi",
                "_score": null,
                "_source": {
                    "username": "username999",
                    "first_name": "Hali",
                    "last_name": "Worboy",
                    "email": "hworboyrq@wix.com",
                    "gender": "Female",
                    "birth_date": "1988-06-02",
                    "address": {
                        "street": "2988 Delladonna Street",
                        "city": "Tallahassee",
                        "province": "Florida",
                        "country": "United States",
                        "zip_code

## Term Query

You need an efficient way to search for exact values (like usernames, product IDs, prices, etc.) in your data. However, using the `Term Query` for text data types might lead to inaccurate results due to text analysis (like automatic lowercasing, symbol removal, etc.).

Instead of using `Term Query` for text data types, it's more appropriate to use `Match Query` as it can handle the peculiarities of text analysis and provide more accurate results.

In [ ]:
# select * from customers where gender = 'Female' limit 10 offset 0 order by username desc
# POST http://localhost:9200/customers/_search
response = es.search(index='customers', body={
    'query': {
        'term': {
            'gender.keyword': 'Female'
        }
    },
    'size': 10,
    'from': 0,
    'sort': [
        {
            "username.keyword": {
                "order": "desc"
            }
        }
    ]
})
print(json.dumps(response.body, indent=4))

{
    "took": 4,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 484,
            "relation": "eq"
        },
        "max_score": null,
        "hits": [
            {
                "_index": "customers",
                "_id": "9rJGYowBtsOVXdO2sMWi",
                "_score": null,
                "_source": {
                    "username": "username999",
                    "first_name": "Hali",
                    "last_name": "Worboy",
                    "email": "hworboyrq@wix.com",
                    "gender": "Female",
                    "birth_date": "1988-06-02",
                    "address": {
                        "street": "2988 Delladonna Street",
                        "city": "Tallahassee",
                        "province": "Florida",
                        "country": "United States",
                        "zip_code"

In [ ]:
# select * from customers where banks.name = 'BCA' limit 10 offset 0 order by username desc
# POST http://localhost:9200/customers/_search
response = es.search(index='customers', body={
    'query': {
        'term': {
            'banks.name.keyword': 'BCA'
        }
    },
    'size': 10,
    'from': 0,
    'sort': [
        {
            "username.keyword": {
                "order": "desc"
            }
        }
    ]
})
print(json.dumps(response.body, indent=4))

{
    "took": 4,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 192,
            "relation": "eq"
        },
        "max_score": null,
        "hits": [
            {
                "_index": "customers",
                "_id": "8rJGYowBtsOVXdO2sMWi",
                "_score": null,
                "_source": {
                    "username": "username997",
                    "first_name": "Christabella",
                    "last_name": "Quincey",
                    "email": "cquinceyro@epa.gov",
                    "gender": "Female",
                    "birth_date": "1984-02-27",
                    "address": {
                        "street": "5697 Arapahoe Trail",
                        "city": "Washington",
                        "province": "District of Columbia",
                        "country": "United States",
               

## Match Query

You're working with text data types, and `Term Query` doesn't provide accurate results because it doesn't use the same text analysis methods.

Use `Match Query`, which, unlike `Term Query`, incorporates the same text analysis as the attribute being searched. Therefore, it's suitable for text data types and can automatically apply the correct text analysis. Though, it's worth noting that `Match Query` can be used for all data types. The topic of text analysis is quite complex and will be covered in a separate module.

In [ ]:
# select * from customers where banks.name = 'BCA' limit 10 offset 0 order by username desc
# POST http://localhost:9200/customers/_search
response = es.search(index='customers', body={
    'query': {
        'match': {
            'banks.name.keyword': 'BCA'
        }
    },
    'size': 10,
    'from': 0,
    'sort': [
        {
            "username.keyword": {
                "order": "desc"
            }
        }
    ]
})
print(json.dumps(response.body, indent=4))

{
    "took": 3,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 192,
            "relation": "eq"
        },
        "max_score": null,
        "hits": [
            {
                "_index": "customers",
                "_id": "8rJGYowBtsOVXdO2sMWi",
                "_score": null,
                "_source": {
                    "username": "username997",
                    "first_name": "Christabella",
                    "last_name": "Quincey",
                    "email": "cquinceyro@epa.gov",
                    "gender": "Female",
                    "birth_date": "1984-02-27",
                    "address": {
                        "street": "5697 Arapahoe Trail",
                        "city": "Washington",
                        "province": "District of Columbia",
                        "country": "United States",
               

### Standard Analyzer

For text data types, by default, the `StandardAnalyzer` is used. Simplistically, this is an object used to convert the original value into a searchable form of smaller tokens.

Here is the official documentation: [StandardAnalyzer](https://lucene.apache.org/core/9_7_0/core/org/apache/lucene/analysis/standard/StandardAnalyzer.html)

- **Example:** When you use the text value `aditira@example.com`, the `StandardAnalyzer` changes it into `aditira example com`. This automatic process allows you to search using the words `aditira`, `example`, or `com`.

### Match Query Operator

When using `Match Query`, there's an attribute called operator that can take the values `OR` or `AND`. By default, the operator value is `OR`.

- **Example:** Therefore, when we search for `bca digital`, the `StandardAnalyzer` will break it down into `bca` OR `digital`. As a result, all banks with 'BCA' or 'BCA Digital' will appear in the search results.

- **Adjustment:** If you want to search for records that contain both `bca` AND `digital`, you need to change the query operator to `AND`.

In [ ]:
# search customers
# POST http://localhost:9200/customers/_search
response = es.search(index='customers', body={
    'query': {
        'match': {
            'banks.name': {
                'query': 'bca digital',
                'operator': 'AND'
            }
        }
    },
    'size': 10,
    'from': 0,
    'sort': [
        {
            "username.keyword": {
                "order": "desc"
            }
        }
    ]
})
print(json.dumps(response.body, indent=4))

{
    "took": 3,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 217,
            "relation": "eq"
        },
        "max_score": null,
        "hits": [
            {
                "_index": "customers",
                "_id": "9rJGYowBtsOVXdO2sMWi",
                "_score": null,
                "_source": {
                    "username": "username999",
                    "first_name": "Hali",
                    "last_name": "Worboy",
                    "email": "hworboyrq@wix.com",
                    "gender": "Female",
                    "birth_date": "1988-06-02",
                    "address": {
                        "street": "2988 Delladonna Street",
                        "city": "Tallahassee",
                        "province": "Florida",
                        "country": "United States",
                        "zip_code"

## Terms Query

- Elasticsearch includes a query that functions similarly to the `IN` operator in SQL, which is the `Terms Query`.
- However, remember that Terms Query behaves much like `Term Query`, so it will not use the default Text Analysis of its attributes.
- So, just like `Term Query`, `Terms Query` is also best suited for attributes that are not of Text data type.

In [ ]:
# search customers
# POST http://localhost:9200/customers/_search
response = es.search(index='customers', body={
    'query': {
        'terms': {
            'username': [
                'username1',
                'username2',
                'username3',
            ]
        }
    },
})
print(json.dumps(response.body, indent=4))

{
    "took": 2,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 3,
            "relation": "eq"
        },
        "max_score": 1.0,
        "hits": [
            {
                "_index": "customers",
                "_id": "KrJGYowBtsOVXdO2r74I",
                "_score": 1.0,
                "_source": {
                    "username": "username1",
                    "first_name": "Rollie",
                    "last_name": "Farge",
                    "email": "rfarge0@sphinn.com",
                    "gender": "Male",
                    "birth_date": "1984-11-23",
                    "address": {
                        "street": "227 Eastwood Pass",
                        "city": "New York City",
                        "province": "New York",
                        "country": "United States",
                        "zip_code": "10131

## Boolean Query

What if we want to search multiple fields? `Term` and `Match` are used to search a single field. If we want to search multiple fields at once, we can combine them using a `Boolean Query`.

### Boolean Query Field

| Occur | Description |
|----------|----------------------------------------------------------------------|
| must | The query has to appear in the resulting document and contribute to the score |
| filter | The query must appear in the resulting document, but does not contribute to the score |
| must_not | The query should not appear in the resulting document |
| should | The query may appear (but is not required) in the resulting document |

Boolean Query - Must

In [ ]:
# search customers
# POST http://localhost:9200/customers/_search
response = es.search(index='customers', body={
    'query': {
        'bool': {
            'must': [
                {
                    'term': {
                        'hobbies': 'gaming'
                    }
                },
                {
                    'term': {
                        'banks.name': 'bca'
                    }
                }
            ]
        }
    },
})
print(json.dumps(response.body, indent=4))

{
    "took": 4,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 106,
            "relation": "eq"
        },
        "max_score": 2.405724,
        "hits": [
            {
                "_index": "customers",
                "_id": "grJGYowBtsOVXdO2r74I",
                "_score": 2.405724,
                "_source": {
                    "username": "username45",
                    "first_name": "Durante",
                    "last_name": "Plampin",
                    "email": "dplampin18@tumblr.com",
                    "gender": "Male",
                    "birth_date": "1978-09-21",
                    "address": {
                        "street": "2132 Homewood Avenue",
                        "city": "New York City",
                        "province": "New York",
                        "country": "United States",
                    

Boolean Query - Filter

In [ ]:
# search customers
# POST http://localhost:9200/customers/_search
response = es.search(index='customers', body={
    'query': {
        'bool': {
            'filter': [
                {
                    'term': {
                        'hobbies': 'gaming'
                    }
                },
                {
                    'term': {
                        'banks.name': 'bca'
                    }
                }
            ]
        }
    },
})
print(json.dumps(response.body, indent=4))

{
    "took": 3,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 106,
            "relation": "eq"
        },
        "max_score": 0.0,
        "hits": [
            {
                "_index": "customers",
                "_id": "MLJGYowBtsOVXdO2r74I",
                "_score": 0.0,
                "_source": {
                    "username": "username4",
                    "first_name": "Jamal",
                    "last_name": "Habard",
                    "email": "jhabard3@yelp.com",
                    "gender": "Male",
                    "birth_date": "1977-10-29",
                    "address": {
                        "street": "01 Toban Place",
                        "city": "Schenectady",
                        "province": "New York",
                        "country": "United States",
                        "zip_code": "12305"
  

Boolean Query - Must Not

In [ ]:
# search customers
# POST http://localhost:9200/customers/_search
response = es.search(index='customers', body={
    'query': {
        'bool': {
            'must_not': [
                {
                    'term': {
                        'hobbies': 'gaming'
                    }
                },
                {
                    'term': {
                        'banks.name': 'bca'
                    }
                }
            ]
        }
    },
})
print(json.dumps(response.body, indent=4))

{
    "took": 3,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 1412,
            "relation": "eq"
        },
        "max_score": 0.0,
        "hits": [
            {
                "_index": "customers",
                "_id": "KbJGYowBtsOVXdO2r74I",
                "_score": 0.0,
                "_source": {
                    "index": {
                        "_index": "customers",
                        "_id": "username1"
                    }
                }
            },
            {
                "_index": "customers",
                "_id": "K7JGYowBtsOVXdO2r74I",
                "_score": 0.0,
                "_source": {
                    "index": {
                        "_index": "customers",
                        "_id": "username2"
                    }
                }
            },
            {
                "_

Boolean Query - Should

In [ ]:
# search customers
# POST http://localhost:9200/customers/_search
response = es.search(index='customers', body={
    'query': {
        'bool': {
            'should': [
                {
                    'term': {
                        'banks.name': 'bca'
                    }
                },
                {
                    'term': {
                        'banks.name': 'bri'
                    }
                }
            ]
        }
    },
})
print(json.dumps(response.body, indent=4))

{
    "took": 5,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 538,
            "relation": "eq"
        },
        "max_score": 2.6825786,
        "hits": [
            {
                "_index": "customers",
                "_id": "vLJGYowBtsOVXdO2sMWi",
                "_score": 2.6825786,
                "_source": {
                    "username": "username970",
                    "first_name": "Terrye",
                    "last_name": "Greatbanks",
                    "email": "tgreatbanksqx@cmu.edu",
                    "gender": "Female",
                    "birth_date": "1971-07-18",
                    "address": {
                        "street": "43271 Sunfield Park",
                        "city": "San Francisco",
                        "province": "California",
                        "country": "United States",
            

### Minimum Should Match

- What happens when we use a `Boolean Query` and only use `should`, without `must` and `filter`? By default, at least one condition within `should` has to match with the document.
- What happens when we add `must` or `filter` to our `Boolean Query`? The minimum `should` match value becomes 0, which means `should` is not mandatory.
- What if we want to change the minimum `should` match value? We have to increase the value for the minimum `should` match.

Boolean Query - Must dan Should

In [ ]:
# POST http://localhost:9200/customers/_search
response = es.search(index='customers', body={
    'query': {
        'bool': {
            'must': {
                'term': {
                    'hobbies': 'gaming'
                }
            },
            'should': [
                {
                    'term': {
                        'banks.name': 'bca'
                    }
                },
                {
                    'term': {
                        'banks.name': 'bri'
                    }
                }
            ]
        }
    }
})
print(json.dumps(response.body, indent=4))

{
    "took": 3,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 321,
            "relation": "eq"
        },
        "max_score": 3.6790433,
        "hits": [
            {
                "_index": "customers",
                "_id": "cLJGYowBtsOVXdO2r74I",
                "_score": 3.6790433,
                "_source": {
                    "username": "username36",
                    "first_name": "Catlee",
                    "last_name": "Ivy",
                    "email": "civyz@tripadvisor.com",
                    "gender": "Female",
                    "birth_date": "1994-08-24",
                    "address": {
                        "street": "64347 7th Parkway",
                        "city": "New Orleans",
                        "province": "Louisiana",
                        "country": "United States",
                        "

Boolean Query - Minimum Should Match

In [ ]:
# POST http://localhost:9200/customers/_search
response = es.search(index='customers', body={
    'query': {
        'bool': {
            'must': {
                'term': {
                    'hobbies': 'gaming'
                }
            },
            'should': [
                {
                    'term': {
                        'banks.name': 'bca'
                    }
                },
                {
                    'term': {
                        'banks.name': 'bri'
                    }
                }
            ],
            'minimum_should_match': 1
        }
    }
})
print(json.dumps(response.body, indent=4))

{
    "took": 3,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 159,
            "relation": "eq"
        },
        "max_score": 3.6790433,
        "hits": [
            {
                "_index": "customers",
                "_id": "cLJGYowBtsOVXdO2r74I",
                "_score": 3.6790433,
                "_source": {
                    "username": "username36",
                    "first_name": "Catlee",
                    "last_name": "Ivy",
                    "email": "civyz@tripadvisor.com",
                    "gender": "Female",
                    "birth_date": "1994-08-24",
                    "address": {
                        "street": "64347 7th Parkway",
                        "city": "New Orleans",
                        "province": "Louisiana",
                        "country": "United States",
                        "

Boolean Query - Minimum Should Match 2

In [ ]:
# POST http://localhost:9200/customers/_search
response = es.search(index='customers', body={
    'query': {
        'bool': {
            'must': {
                'term': {
                    'hobbies': 'gaming'
                }
            },
            'should': [
                {
                    'term': {
                        'banks.name': 'bca'
                    }
                },
                {
                    'term': {
                        'banks.name': 'bri'
                    }
                }
            ],
            'minimum_should_match': 2
        }
    }
})
print(json.dumps(response.body, indent=4))

{
    "took": 3,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 19,
            "relation": "eq"
        },
        "max_score": 3.6790433,
        "hits": [
            {
                "_index": "customers",
                "_id": "cLJGYowBtsOVXdO2r74I",
                "_score": 3.6790433,
                "_source": {
                    "username": "username36",
                    "first_name": "Catlee",
                    "last_name": "Ivy",
                    "email": "civyz@tripadvisor.com",
                    "gender": "Female",
                    "birth_date": "1994-08-24",
                    "address": {
                        "street": "64347 7th Parkway",
                        "city": "New Orleans",
                        "province": "Louisiana",
                        "country": "United States",
                        "z

## Cat API

Imagine Elasticsearch as a bustling city filled with information. The Compact and Aligned Text (CAT) APIs are like the city's information kiosks, where you can get a snapshot of the city's health, districts (indices), buildings (shards), citizens (nodes), and more.

These CAT APIs are not typically used for day-to-day city life (the applications we build), but rather for city planners and maintenance crews (users maintaining Elasticsearch).

There is a vast network of these information kiosks (APIs) throughout the city. You can see them all by visiting the city's main information center (GET /_cat). If you'd like to learn more about these kiosks, you can visit the city's online guidebook at [Elasticsearch's CAT APIs documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/cat.html ).

In [ ]:
import requests

response = requests.get('http://localhost:9200/_cat')
print(response.text)

=^.^=
/_cat/allocation
/_cat/shards
/_cat/shards/{index}
/_cat/master
/_cat/nodes
/_cat/tasks
/_cat/indices
/_cat/indices/{index}
/_cat/segments
/_cat/segments/{index}
/_cat/count
/_cat/count/{index}
/_cat/recovery
/_cat/recovery/{index}
/_cat/health
/_cat/pending_tasks
/_cat/aliases
/_cat/aliases/{alias}
/_cat/thread_pool
/_cat/thread_pool/{thread_pools}
/_cat/plugins
/_cat/fielddata
/_cat/fielddata/{fields}
/_cat/nodeattrs
/_cat/repositories
/_cat/snapshots/{repository}
/_cat/templates
/_cat/component_templates/_cat/ml/anomaly_detectors
/_cat/ml/anomaly_detectors/{job_id}
/_cat/ml/datafeeds
/_cat/ml/datafeeds/{datafeed_id}
/_cat/ml/trained_models
/_cat/ml/trained_models/{model_id}
/_cat/ml/data_frame/analytics
/_cat/ml/data_frame/analytics/{id}
/_cat/transforms
/_cat/transforms/{transform_id}



In [ ]:
response = requests.get('http://localhost:9200/_cat/health')
print(response.text)

1702458158 09:02:38 ai-bootcamp yellow 1 1 3 3 0 0 3 0 - 50.0%



In [ ]:
response = requests.get('http://localhost:9200/_cat/indices')
print(response.text)

yellow open orders    iZ5yo0jhQzen6WpHWu02pw 1 1    1 0   6.6kb   6.6kb
yellow open customers DvA3r0qSTxSDO5lJaQNquQ 1 1 2000 0 555.7kb 555.7kb
yellow open products  1cdU4XybTUad8p0TyqKtlQ 1 1    5 0   5.5kb   5.5kb

